In [2]:
import pandas as pd
from MLP import *
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SVMSMOTE

In [3]:
df = pd.read_csv("neo_v2.csv")
df = df.sample(1000, random_state= 42)
df.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
20406,3943344,(2019 YT1),0.024241,0.054205,22148.962596,5.028574e+07,Earth,False,25.20,False
74443,3879239,(2019 US),0.012722,0.028447,26477.211836,1.683201e+06,Earth,False,26.60,False
74306,3879244,(2019 UU),0.013322,0.029788,33770.201397,3.943220e+06,Earth,False,26.50,False
45943,2481965,481965 (2009 EB1),0.193444,0.432554,43599.575296,7.346837e+07,Earth,False,20.69,False
62859,3789471,(2017 WJ1),0.044112,0.098637,36398.080883,6.352916e+07,Earth,False,23.90,False


In [4]:
df.dtypes

id                      int64
name                   object
est_diameter_min      float64
est_diameter_max      float64
relative_velocity     float64
miss_distance         float64
orbiting_body          object
sentry_object            bool
absolute_magnitude    float64
hazardous                bool
dtype: object

In [5]:
X = df.iloc[:, 2:6]
x = df.iloc[:,8].values
x = x.reshape((-1,1))
X = np.append(X,x,axis = 1)
y = df.iloc[:, -1].values
y = y.reshape((-1, 1)).astype(int)
sc= StandardScaler()
X = sc.fit_transform(X)

In [6]:
X.shape

(1000, 5)

In [7]:
smote = SVMSMOTE()
X, y = smote.fit_resample(X, y)
y = y.reshape((-1, 1)).astype(int)

In [8]:
X.shape

(1822, 5)

In [9]:
topology = np.array([5,16,8,1])
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [10]:
mlp = MultiLayerPerceptron(topology, "meansquared", "sigmoid")
mlp.learn(x_train, y_train, .1, 1000, batch_size=10, printSummary=True)

100%|████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 199.86it/s, loss=0.0106, accuracy=1]

---------------------------------
Structure: 
Layer 0:
  Number of Neurons: 5

Layer 1:
  Number of Neurons: 16
  Activation Function: Sigmoid

Layer 2:
  Number of Neurons: 8
  Activation Function: Sigmoid

Layer 3:
  Number of Neurons: 1
  Activation Function: Sigmoid

Loss Function: MeanSquaredLossFunction
Learningrate: 0.1
Epochs: 1000
Loss: 0.010568547628551254
Accuracy: 1.0
---------------------------------


In [12]:
yPred = mlp.predictAll(x_test,False)
yPred = np.round(yPred)
yPred = (yPred== y_test)
accuracy = np.count_nonzero(yPred == True)/len(yPred)
accuracy*100

86.69410150891632